## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-09-18-55-20 +0000,nypost,Gang shooting outside Ghislaine Maxwell’s new ...,https://nypost.com/2025/08/09/us-news/gang-sho...
1,2025-08-09-18-42-26 +0000,nyt,A Sidelined Europe Seeks a Voice as Trump and ...,https://www.nytimes.com/2025/08/09/world/europ...
2,2025-08-09-18-37-10 +0000,nypost,Coyote attacks little boy at Southern Californ...,https://nypost.com/2025/08/09/us-news/boy-atta...
3,2025-08-09-18-26-37 +0000,cbc,London police arrest hundreds as pro-Palestini...,https://www.cbc.ca/news/world/uk-london-protes...
4,2025-08-09-18-14-42 +0000,nyt,"Morton Meyerson, Texas Businessman and Perot L...",https://www.nytimes.com/2025/08/09/business/mo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,59
5,new,17
15,putin,14
361,gaza,14
47,ukraine,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
148,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...,148
6,2025-08-09-18-03-49 +0000,nypost,"Ukraine, European leaders counter Russian ceas...",https://nypost.com/2025/08/09/world-news/ukrai...,123
69,2025-08-09-12-09-19 +0000,cbc,Zelenskyy says any peace deal without Ukraine ...,https://www.cbc.ca/news/world/zelenskyy-dismis...,119
86,2025-08-09-09-48-43 +0000,nypost,Zelensky vows Russia will not get any Ukrainia...,https://nypost.com/2025/08/09/us-news/zelensky...,118
173,2025-08-08-23-04-25 +0000,cbc,Trump says he'll meet Putin in Alaska next Fri...,https://www.cbc.ca/news/world/trump-putin-meet...,116


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
148,148,2025-08-09-01-00-00 +0000,wsj,President Trump is considering reclassifying m...,https://www.wsj.com/politics/policy/trump-mari...
6,64,2025-08-09-18-03-49 +0000,nypost,"Ukraine, European leaders counter Russian ceas...",https://nypost.com/2025/08/09/world-news/ukrai...
94,50,2025-08-09-09-02-47 +0000,nyt,Israel’s New Plan to Take Gaza City,https://www.nytimes.com/video/world/1000000103...
10,45,2025-08-09-17-35-41 +0000,bbc,Atlanta police officer dies after shooting nea...,https://www.bbc.com/news/articles/cp3ekez0k2xo...
194,43,2025-08-08-21-46-42 +0000,cbc,"Azerbaijan, Armenia sign U.S.-brokered peace d...",https://www.cbc.ca/news/world/trump-azerbaijan...
173,43,2025-08-08-23-04-25 +0000,cbc,Trump says he'll meet Putin in Alaska next Fri...,https://www.cbc.ca/news/world/trump-putin-meet...
161,35,2025-08-08-23-49-00 +0000,wsj,President Trump removed former congressman Bil...,https://www.wsj.com/politics/policy/billy-long...
13,34,2025-08-09-17-07-56 +0000,nypost,Financial expert warns young Americans against...,https://nypost.com/2025/08/09/business/financi...
39,31,2025-08-09-15-03-32 +0000,nypost,Drunk illegal immigrant driver was 3 times ove...,https://nypost.com/2025/08/09/us-news/new-char...
86,31,2025-08-09-09-48-43 +0000,nypost,Zelensky vows Russia will not get any Ukrainia...,https://nypost.com/2025/08/09/us-news/zelensky...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
